In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/07 18:26:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [5]:
from datetime import datetime

In [11]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [14]:
rows = rdd.take(10)
row = rows[0]

In [15]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)

In [17]:
row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)

datetime.datetime(2020, 1, 23, 13, 0)

In [18]:
def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)
    
    return (key, value)

In [22]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)    

In [27]:
from collections import namedtuple

In [28]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [29]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0],
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [33]:
from pyspark.sql import types

In [34]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [35]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [39]:
df_result.schema

StructType([StructField('hour', TimestampType(), True), StructField('zone', IntegerType(), True), StructField('revenue', DoubleType(), True), StructField('count', IntegerType(), True)])

In [38]:
df_result.write.parquet('tmp/green-revenue')

In [43]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [50]:
import pandas as pd

In [51]:
rows = duration_rdd.take(10)

In [54]:
pd.DataFrame(rows, columns=columns)

,VendorID,lpep_pickup_datetime,PULocationID,DOLocationID,trip_distance
0,2.0,2020-01-23 13:10:15,74,130,12.77
1,NaN,2020-01-20 15:09:00,67,39,8.00
2,2.0,2020-01-15 20:23:41,260,157,1.27
3,2.0,2020-01-05 16:32:26,82,83,1.25
4,2.0,2020-01-29 19:22:42,166,42,1.84
5,2.0,2020-01-15 11:07:42,179,223,0.76
6,2.0,2020-01-16 08:22:29,41,237,3.32
7,2.0,2020-01-28 17:05:28,75,161,2.21
8,1.0,2020-01-22 14:51:37,152,166,0.90
9,2.0,2020-01-31 10:25:04,75,234,6.10


In [61]:
# models = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred


In [62]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions
    
    for row in df.itertuples():
        yield row

In [67]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch) \
    .toDF() \
    .drop('Index')

In [68]:
df_predicts.show()

+--------+--------------------+------------+------------+-------------+------------------+
|VendorID|lpep_pickup_datetime|PULocationID|DOLocationID|trip_distance|predicted_duration|
+--------+--------------------+------------+------------+-------------+------------------+
|     2.0|                  {}|          74|         130|        12.77|63.849999999999994|
|     NaN|                  {}|          67|          39|          8.0|              40.0|
|     2.0|                  {}|         260|         157|         1.27|              6.35|
|     2.0|                  {}|          82|          83|         1.25|              6.25|
|     2.0|                  {}|         166|          42|         1.84| 9.200000000000001|
|     2.0|                  {}|         179|         223|         0.76|               3.8|
|     2.0|                  {}|          41|         237|         3.32|16.599999999999998|
|     2.0|                  {}|          75|         161|         2.21|             11.05|